# Natural Language Processing to SQL
*( How to make GPT-2 Multi-Tasking )* 🤗

In [1]:
!pip install transformers accelerate datasets xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd

import re
import os

import logging

In [3]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG) #INFO

In [4]:
import torch
from datasets import load_dataset

In [5]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [7]:
template = """
<question> {{question}}

<query> {{query}} </query>
<params> {{params}} </params>
<comment> {{comment}} </comment>

</question>
"""

print(template)


<question> {{question}}

<query> {{query}} </query>
<params> {{params}} </params>
<comment> {{comment}} </comment>

</question>



In [6]:
ds = load_dataset('text', data_dir='/gdrive/MyDrive/Datasets/NLP/Comments_Generator/template_total_dataset/', download_mode=False, streaming=False)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [8]:
print(ds['train'][0]['text'])

<question> nell'anno 2019, i prodotti mele, pesche da chi sono stati comprati <query> SELECT DISTINCT cli FROM tabella WHERE (anno = 2019 AND prod IN ('mele', 'pesche')) </query> <params> </params> <comment> </comment> </question> 


In [10]:
print(ds['train'][-1]['text'])

<question> il prodotto più venduto nell'anno 2017 <query> </query> <params> fragole, 53 </params> <comment> il prodotto più venduto del 2017 sono le fragole </comment> </question> 


In [11]:
from transformers import AutoTokenizer, GPT2Tokenizer

In [12]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

tokenizer.__dict__

{'init_inputs': (),
 'init_kwargs': {'errors': 'replace',
  'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True),
  'bos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True),
  'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True),
  'pad_token': None,
  'add_prefix_space': False,
  'add_bos_token': False,
  'model_max_length': 1024,
  'special_tokens_map_file': None,
  'name_or_path': 'gpt2'},
 'name_or_path': 'gpt2',
 '_processor_class': None,
 'model_max_length': 1024,
 'padding_side': 'right',
 'truncation_side': 'right',
 'model_input_names': ['input_ids', 'attention_mask'],
 'clean_up_tokenization_spaces': True,
 'deprecation_warnings': {},
 '_in_target_context_manager': False,
 '_bos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True),
 '_eos_token': AddedToken("<|endoftext|>", rstrip

In [13]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2",
                                          bos_token = '<question>',
                                          eos_token = '</question>',
                                          pad_token = '<|endoftext|>',
                                          pad_token_id = 50256)


tokenizer.add_tokens(['<query>', '</query>', '<params>', '</params>',
                                   '<comment>', '</comment>'])


tokenizer.add_tokens(['DISTINCT','strftime'])

tokenizer.__dict__

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{'init_inputs': (),
 'init_kwargs': {'errors': 'replace',
  'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True),
  'bos_token': AddedToken("<question>", rstrip=False, lstrip=False, single_word=False, normalized=True),
  'eos_token': AddedToken("</question>", rstrip=False, lstrip=False, single_word=False, normalized=True),
  'pad_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True),
  'add_prefix_space': False,
  'add_bos_token': False,
  'pad_token_id': 50256,
  'model_max_length': 1024,
  'special_tokens_map_file': None,
  'name_or_path': 'gpt2'},
 'name_or_path': 'gpt2',
 '_processor_class': None,
 'model_max_length': 1024,
 'padding_side': 'right',
 'truncation_side': 'right',
 'model_input_names': ['input_ids', 'attention_mask'],
 'clean_up_tokenization_spaces': True,
 'deprecation_warnings': {},
 '_in_target_context_manager': False,
 '_bos_token': AddedToken("<question>", rstrip=Fal

In [ ]:
# assert tokenizer.encoder['<|endoftext|>'] == 50256

In [14]:
# max(len(tokenizer.tokenize(text['text'])) for text in ds['train'])

In [15]:
# max(len(tokenizer.tokenize(text['text'])) for text in ds['test'])

In [16]:
def preprocess_for_tokenizer(txt:str):
  return tokenizer(txt['text'], truncation=True, max_length=100, padding='max_length')

In [17]:
train_dataset = ds['train'].map(preprocess_for_tokenizer).shuffle(seed=42)
test_dataset = ds['test'].map(preprocess_for_tokenizer).shuffle(seed=42)

Map:   0%|          | 0/86696 [00:00<?, ? examples/s]

Map:   0%|          | 0/28900 [00:00<?, ? examples/s]

In [18]:
print(train_dataset[0].keys())

dict_keys(['text', 'input_ids', 'attention_mask'])


In [19]:
train_dataset[0]['text']

"<question> nell'anno 2019 chi ha comprato: kiwi, pere, pesche <query> SELECT DISTINCT cli FROM tabella WHERE (anno = 2019 AND prod IN ('kiwi', 'pere', 'pesche')) </query> <params> </params> <comment> </comment> </question> "

In [20]:
print(tokenizer.tokenize(train_dataset[0]['text']))

['<question>', 'Ġne', 'll', "'", 'ann', 'o', 'Ġ2019', 'Ġchi', 'Ġha', 'Ġcompr', 'ato', ':', 'Ġk', 'iw', 'i', ',', 'Ġp', 'ere', ',', 'Ġpes', 'che', '<query>', 'SELECT', 'DISTINCT', 'cli', 'ĠFROM', 'Ġtab', 'ella', 'ĠWHERE', 'Ġ(', 'ann', 'o', 'Ġ=', 'Ġ2019', 'ĠAND', 'Ġprod', 'ĠIN', "Ġ('", 'ki', 'wi', "',", "Ġ'", 'pe', 're', "',", "Ġ'", 'pes', 'che', "'", '))', '</query>', '<params>', '</params>', '<comment>', '</comment>', '</question>', 'Ġ']


In [22]:
# tokenizer.eos_token_id

In [25]:
from transformers import Trainer, TrainingArguments, AutoConfig, AutoModelForCausalLM, GPT2LMHeadModel

In [23]:
config = AutoConfig.from_pretrained('gpt2',
                                    bos_token_id=tokenizer.bos_token_id,
                                    eos_token_id=tokenizer.eos_token_id,
                                    pad_token_id=tokenizer.pad_token_id,
                                    output_hidden_states=False)

In [26]:
# model = AutoModelForCausalLM.from_pretrained("gpt2", config=config)
model = GPT2LMHeadModel.from_pretrained("gpt2", config=config)

In [27]:
model.num_parameters

<bound method ModuleUtilsMixin.num_parameters of GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)>

In [28]:
model.resize_token_embeddings(len(tokenizer))

Embedding(50267, 768)

In [29]:
model.num_parameters

<bound method ModuleUtilsMixin.num_parameters of GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50267, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50267, bias=False)
)>

In [31]:
from transformers import DataCollatorForLanguageModeling#, TextDataset

In [33]:
data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False
    )

In [34]:
training_args = TrainingArguments(
    output_dir='/gdrive/MyDrive/temporary_models/NLP_2_SQL', #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=8,  # batch size for evaluation
    eval_steps = 40, # Number of update steps between two evaluations.
    save_steps = 1000, # after # steps model is saved
    warmup_steps=50,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [35]:
# torch.cuda.empty_cache()

#### Resume the training

In [ ]:
resume = True # @param ["False", "True"] {type:"raw"}

In [ ]:
trainer.train(resume_from_checkpoint=resume)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


KeyboardInterrupt: ignored

In [37]:
# import ipywidgets as widgets
# from IPython.display import display
# button = widgets.Button(description="Click Me!")
# output = widgets.Output()

# def on_button_clicked(b):
#   # Display the message within the output widget.
#   with output:
#     print("Button clicked.")

# button.on_click(on_button_clicked)
# display(button, output)

In [38]:
save_or_load_model = "load" #@param ["save", "load"]

In [39]:
path = '/gdrive/MyDrive/Datasets/NLP/Comments_Generator/Template'

if save_or_load_model == 'save':
  trainer.save_model(path)
  tokenizer.save_pretrained(path)

In [40]:
from transformers import pipeline

nlp2query = pipeline('text-generation', model=path, tokenizer=tokenizer)

In [41]:
import jinja2
env = jinja2.Environment()

template = env.from_string(template)

In [42]:
def step_1(prompt:str) -> str:

  """Preparara il prompt per la generazione della query"""

  data = {}

  data['question'] = prompt
  data['query'] = ""
  data['params'] = ""
  data['comment'] = ""

  text = template.render(data)
  text = text.split("\n")
  text = " ".join(text)
  text = text.split("</query>")[0]
  text = text.strip()

  text = re.sub(pattern="[ ]+", repl=" ", string=text)

  return text

In [99]:
prompt = "qual'è il prodotto più venduto nel 2020"

In [100]:
text = step_1(prompt)
text

"<question> qual'è il prodotto più venduto nel 2020 <query>"

In [101]:
prompt_and_query = nlp2query(text, max_length=100)[0]['generated_text']
print(prompt_and_query)

<question> qual'è il prodotto più venduto nel 2020 <query> SELECT prod, SUM(qty) as venduto FROM tabella WHERE anno = 2020 GROUP BY prod ORDER BY 2 DESC LIMIT 1 </query> <params> </params> <comment> </comment>


In [90]:
def step_2(text:str) -> str:
  """Restituisce il testo all'interno dei tag <query> </query>"""

  text = re.findall(pattern=r"<query> (.*) </query>", string=text)[0]

  return text

In [91]:
query = step_2(prompt_and_query)
print(query)

SELECT prod, SUM(qty) as venduto FROM tabella WHERE anno = 2020 GROUP BY prod ORDER BY 2 DESC LIMIT 1


In [70]:
import sqlite3
import re

from os.path import exists

In [49]:
class DBConnection:

    def __init__(self, nome_db:str, NOME_TABELLA:str="tabella"):
        self.nome_db = nome_db
        self.NOME_TABELLA = NOME_TABELLA
        self.con = None
        self.df = None

        self._get_db_connection()

    def _get_db_connection(self):

        """
        Restituisce la connessione del db
        """

        if not exists(self.nome_db):
            raise Exception("Il file non esiste!")

        try:
            self.con = sqlite3.connect(f"{self.nome_db}")
            return True

        except Exception as err:
            raise Exception(err)

    def get_df(self) -> pd.DataFrame:

        """
        Si connette al DB ed estrae la tabella restituendo il dataframe
        """

        try:
            self.df = pd.read_sql(f"SELECT * FROM {self.NOME_TABELLA}", con=self.con)
            return self.df

        except Exception as err:
            logger.error(f"Qualcosa è andato storto con la connessione\n {err}")
            return False

    def run_query(self, query:str):

        cur = self.con.cursor()
        res = cur.execute(query)

        return res.fetchall()

In [71]:
db_path = '/gdrive/MyDrive/Datasets/NLP/Comments_Generator/Template/db_sales.db'

In [72]:
db = DBConnection(db_path)

In [92]:
parameters = db.run_query(query)
parameters

[('arance', 91)]

In [102]:
params = ", ".join(np.array(parameters).reshape(-1))

In [103]:
def step_3(prompt:str, params:str) -> str:

  """Prepara il testo da passare all'algoritmo in modo che contenga il prompt
  e i parametri che saranno usati per generare il commento"""

  data = {}

  prompt = prompt
  params = ", ".join(np.array(params).reshape(-1))

  data['question'] = prompt
  data['query'] = ""
  data['params'] = params
  data['comment'] = ""

  text = template.render(data)
  text = text.split("\n")
  text = " ".join(text)
  text = text.split("</comment>")[0]
  text = text.strip()

  text = re.sub(pattern="[ ]+", repl=" ", string=text)

  return text

In [104]:
text = step_3(prompt, params)
text

"<question> qual'è il prodotto più venduto nel 2020 <query> </query> <params> arance, 91 </params> <comment>"

In [105]:
nlp2query(text, max_length=100)[0]['generated_text']

"<question> qual'è il prodotto più venduto nel 2020 <query> </query> <params> arance, 91 </params> <comment> il prodotto più venduto nell'anno 2020 sono le arance </comment>"

In [106]:
def get_comment(text:str) -> str:
  """Restituisce il testo all'interno dei tag <comment> </comment>"""

  text = re.findall(pattern=r"<comment> (.*) </comment>", string=text)[0]

  return text

In [110]:
def agent(prompt:str) -> str:
  """Riceve il prompt e gestisce tutti gli step fino al commento finale"""

  text = step_1(prompt)
  logger.debug(text)

  print("Inizio generazione della query...")
  text = nlp2query(text, max_length=100)[0]['generated_text']
  logger.debug(text)
  query = step_2(text)
  logger.debug(query)

  print("...estrazione dei parametri...")
  parameters = db.run_query(query)
  logger.debug(parameters)

  print("...generazione del commento.")
  text = step_3(prompt, parameters)
  text = nlp2query(text, max_length=100)[0]['generated_text']
  logger.debug(text)

  response = get_comment(text)

  return response

In [112]:
prompt = "qual'è il prodotto più venduto nell'anno 2020"

risp = agent(prompt)

print(f"\nTesto generato ----> {risp}")

DEBUG:__main__:<question> qual'è il prodotto più venduto nell'anno 2020 <query>


Inizio generazione della query...


DEBUG:__main__:<question> qual'è il prodotto più venduto nell'anno 2020 <query> SELECT prod, SUM(qty) as venduto FROM tabella WHERE anno = 2020 GROUP BY prod ORDER BY 2 DESC LIMIT 1 </query> <params> </params> <comment> </comment>
DEBUG:__main__:SELECT prod, SUM(qty) as venduto FROM tabella WHERE anno = 2020 GROUP BY prod ORDER BY 2 DESC LIMIT 1
DEBUG:__main__:[('arance', 91)]


...estrazione dei parametri...
...generazione del commento.


DEBUG:__main__:<question> qual'è il prodotto più venduto nell'anno 2020 <query> </query> <params> arance, 91 </params> <comment> il prodotto più venduto nell'anno 2020 sono le arance </comment>



Testo generato ----> il prodotto più venduto nell'anno 2020 sono le arance


In [114]:
prompt_and_params = "<question> qual'è il prodotto più venduto nell'anno 2020 <query> </query> <params> arance, 91 </params> <comment>"

In [120]:
nlp2query(prompt_and_params, temperature=1.0, max_length=100)

[{'generated_text': "<question> qual'è il prodotto più venduto nell'anno 2020 <query> </query> <params> arance, 91 </params> <comment> sono le arance il prodotto più venduto nell'anno 2020 </comment>"}]